<img align="right" src="images/tf.png" width="128"/>
<img align="right" src="images/uu-small.png" width="128"/>
<img align="right" src="images/dans.png" width="128"/>

---

To get started: consult [start](start.ipynb)

---

# Rings

Rings are literary structures of text composition, typically at the sub-sura level.
They are marked by certain formal characteristics and a semantic structure where elements at the start
are mirrored semantically at the end.
The resulting effect is that the crux of the narrative/exposition appears at its center.

This is not an easy concept to deal with computationally, because our dataset has only limited semantic information,
and the observed patterns are the result of sensitivity of the human mind to a literary and religious text.

So, do not expect to a recipe to identify rokings, but just a scratching of their surface.

Moreover, this notebook is written from the view-point of a non-expert in Islam and Arabic.
It serves as a testimony of how much perception such a person can muster.
If you, as an expert, read on, it will become clear how much room for improvement there is.
The challenge for you is then to strengthen your computational skills, so that you
can actually make those improvements.

The notebook finishes off by showing how to save, export and share the data you create.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import collections
from IPython.display import Markdown, display

from tf.app import use

In [3]:
# A = use('quran', hoist=globals())
A = use('quran:clone', checkout="clone", hoist=globals())

Using TF-app in /Users/dirk/github/annotation/app-quran/code:
	repo clone offline under ~/github (local github)
Using data in /Users/dirk/github/q-ran/quran/tf/0.4:
	repo clone offline under ~/github (local github)


# Exploration

We follow an article by
Raymond K. Farrin, [*Surat al-Baqara: A Structural Analysis*](https://www.academia.edu/8642515/Surat_al-Baqarah_A_Structural_Analysis)

## Trival detail: names of suras

The layman does not know what the Al-Baqara sura is. But there is data!
Let's print the names of the suras.

In [4]:
for s in F.otype.s('sura'):
  print(f'{F.number.v(s)}: {F.nameTrans.v(s)} ({F.name.v(s)})')

1: Al-Faatiha (الفاتحة)
2: Al-Baqara (البقرة)
3: Aal-i-Imraan (آل عمران)
4: An-Nisaa (النساء)
5: Al-Maaida (المائدة)
6: Al-An'aam (الأنعام)
7: Al-A'raaf (الأعراف)
8: Al-Anfaal (الأنفال)
9: At-Tawba (التوبة)
10: Yunus (يونس)
11: Hud (هود)
12: Yusuf (يوسف)
13: Ar-Ra'd (الرعد)
14: Ibrahim (ابراهيم)
15: Al-Hijr (الحجر)
16: An-Nahl (النحل)
17: Al-Israa (الإسراء)
18: Al-Kahf (الكهف)
19: Maryam (مريم)
20: Taa-Haa (طه)
21: Al-Anbiyaa (الأنبياء)
22: Al-Hajj (الحج)
23: Al-Muminoon (المؤمنون)
24: An-Noor (النور)
25: Al-Furqaan (الفرقان)
26: Ash-Shu'araa (الشعراء)
27: An-Naml (النمل)
28: Al-Qasas (القصص)
29: Al-Ankaboot (العنكبوت)
30: Ar-Room (الروم)
31: Luqman (لقمان)
32: As-Sajda (السجدة)
33: Al-Ahzaab (الأحزاب)
34: Saba (سبإ)
35: Faatir (فاطر)
36: Yaseen (يس)
37: As-Saaffaat (الصافات)
38: Saad (ص)
39: Az-Zumar (الزمر)
40: Al-Ghaafir (غافر)
41: Fussilat (فصلت)
42: Ash-Shura (الشورى)
43: Az-Zukhruf (الزخرف)
44: Ad-Dukhaan (الدخان)
45: Al-Jaathiya (الجاثية)
46: Al-Ahqaf (الأحقاف)
47: Muhammad (محم

The article is about sura 2.

In [5]:
sura2 = F.otype.s('sura')[1]

## Sections

The article speaks about sections in sura2.
Those sections apparently form a level between the sura and the aya.

> The sura, 286 verses, consists of nine sections. 

```
A    1 -  20
B   21 -  39
C   40 - 103
D  104 - 141
E  142 - 152
D' 153 - 177
C' 178 - 253
B' 254 - 284
A' 285 - 286
```

Can we identify those sections in the data?

In [6]:
sections = [
  (1, 20),
  (21, 39),
  (40, 103),
  (104, 141),
  (142, 152),
  (153, 177),
  (178, 253),
  (254, 284),
  (285, 286),
]

nSections = len(sections)
sectionStarts = {b for (b, e) in sections}

Apart from suras and ayas, there are other sectional objects in our data, see the 
feature documentation (one of the links after the incantation):

* juz
* hizb
* manzil
* ruku
* page
* sajda

Let's see how our sura is divided into these units.

Per unit type we list the how many of those units there are in this sura.

In [7]:
UTYPES = '''
juz
hizb
manzil
ruku
page
sajda
'''.strip().split()

In [8]:
for uType in UTYPES:
  units = L.d(sura2, otype=uType)
  print(f'{uType.upper():<6}: {len(units):>2} x')

JUZ   :  1 x
HIZB  : 18 x
MANZIL:  0 x
RUKU  : 40 x
PAGE  : 48 x
SAJDA :  0 x


Now we want to now the exact aya intervals of the hizb, ruku and page units.

In [9]:
for uType in UTYPES:
  units = L.d(sura2, otype=uType)
  if not len(units):
    continue
  print(f'{uType.upper():<6}: {len(units):>2} x')
  for unit in units:
    unitNum = F.number.v(unit)
    ayas = L.d(unit, otype='aya')
    firstAya = F.number.v(ayas[0])
    lastAya = F.number.v(ayas[-1])
    print(f'\t{unitNum:>2}: aya {firstAya:>3}-{lastAya:>3}')

JUZ   :  1 x
	 2: aya 142-252
HIZB  : 18 x
	 2: aya  26- 43
	 3: aya  44- 59
	 4: aya  60- 74
	 5: aya  75- 91
	 6: aya  92-105
	 7: aya 106-123
	 8: aya 124-141
	 9: aya 142-157
	10: aya 158-176
	11: aya 177-188
	12: aya 189-202
	13: aya 203-218
	14: aya 219-232
	15: aya 233-242
	16: aya 243-252
	17: aya 253-262
	18: aya 263-271
	19: aya 272-282
RUKU  : 40 x
	 2: aya   1-  7
	 3: aya   8- 20
	 4: aya  21- 29
	 5: aya  30- 39
	 6: aya  40- 46
	 7: aya  47- 59
	 8: aya  60- 61
	 9: aya  62- 71
	10: aya  72- 82
	11: aya  83- 86
	12: aya  87- 96
	13: aya  97-103
	14: aya 104-112
	15: aya 113-121
	16: aya 122-129
	17: aya 130-141
	18: aya 142-147
	19: aya 148-152
	20: aya 153-163
	21: aya 164-167
	22: aya 168-176
	23: aya 177-182
	24: aya 183-188
	25: aya 189-196
	26: aya 197-210
	27: aya 211-216
	28: aya 217-221
	29: aya 222-228
	30: aya 229-231
	31: aya 232-235
	32: aya 236-242
	33: aya 243-248
	34: aya 249-253
	35: aya 254-257
	36: aya 258-260
	37: aya 261-266
	38: aya 267-273
	39: aya 

It seems that the section boundaries all coincide with ruku boundaries.

Let's check the degree in which the section boundaries respect the unit boundaries, for each unit type.

We collect the start ayas for each unit.

In [10]:
startAyas = collections.defaultdict(set)

for uType in UTYPES:
  units = L.d(sura2, otype=uType)
  if not len(units):
    continue
  for unit in units:
    unitNum = F.number.v(unit)
    ayas = L.d(unit, otype='aya')
    firstAya = F.number.v(ayas[0])
    startAyas[uType].add(firstAya)

Now we count how many section starts are members of the startAyas for each unit type. 

In [11]:
agreement = {}

for uType in UTYPES:
  units = L.d(sura2, otype=uType)
  if not len(units):
    continue
  agreement[uType] = len(startAyas[uType] & sectionStarts)
  
for (uType, agreement) in agreement.items():
  print(f'{agreement} out of {nSections} section starts coincide with a {uType} start')

1 out of 9 section starts coincide with a juz start
1 out of 9 section starts coincide with a hizb start
7 out of 9 section starts coincide with a ruku start
2 out of 9 section starts coincide with a page start


The rukus are the best match, but two sections cross a ruku boundary:

In [12]:
sorted(sectionStarts - startAyas['ruku'])

[178, 285]

## First and last lines

We explore the first and last ayas of each section.

We make a list of tuples of their nodes.

Note that the number of an aya is not the same as the node number of the aya.

Node numbers are like barcodes: they identify objects uniquely within the whole universe.
Aya numbers only identify an aya within its sura.
But no two ayas have the same node number.

All functions of TF require node numbers, so every now and then we have to move from the
identifications in front of our nose to the underlying node numbers.

That is what the `T.nodeFromSection()` does: it takes a pair (sura number, aya number) and
converts it into a unique aya node number.

In [13]:
boundaryAyaNodes = []

for (beginAnum, endAnum) in sections:
  beginA = T.nodeFromSection((2, beginAnum))
  endA = T.nodeFromSection((2, endAnum))
  boundaryAyaNodes.append((beginA, endA))

Here is the tuple we constructed: an abstract ensemble of barcodes:

In [14]:
boundaryAyaNodes

[(128227, 128246),
 (128247, 128265),
 (128266, 128329),
 (128330, 128367),
 (128368, 128378),
 (128379, 128403),
 (128404, 128479),
 (128480, 128510),
 (128511, 128512)]

Now we ask TF to show them in a more insightful way:

In [15]:
A.table(boundaryAyaNodes)

But I do not read Arabic, and I want to get a sense of what is going on.

We display them more extensively by using `A.show()` instead. 
Before we make the call, we set up a
[display parameter](https://annotation.github.io/text-fabric/Api/App/#display)
that calls up the `translation@en` feature for each aya.

In [20]:
A.displaySetup(extraFeatures='translation@en', prettyTypes=False)

If we run `A.show()` now, we get a display of ayas and words with translations and morphological information.

In [21]:
A.show(boundaryAyaNodes, baseTypes="group")

## Vocative and resumption particles

With my unaided eye I can already see that the start aya tends to start with a vocative particle and the end aya with a resumption particle.

We now build a table of all ayas in this sura, where we highlight first words if they are a resumption or vocative particle.
We also highlight the ayas that are the start and end ayas of the sections.

### Building a tuple of the objects of interest

We proceed by building a list of aya nodes and their first words.

We also make sets of the ayas where a section starts and where a section ends.

In [22]:
sura2Ayas = L.d(sura2, otype='aya')

ayaTuples = [(a, L.d(a, otype='word')[0]) for a in sura2Ayas]

startAyaNodes = set()
endAyaNodes = set()

for (beginAnum, endAnum) in sections:
  beginA = T.nodeFromSection((2, beginAnum))
  endA = T.nodeFromSection((2, endAnum))
  startAyaNodes.add(beginA)
  endAyaNodes.add(endA)

### Defining highlights

We can make a dictionary of nodes and the colors we want to display their objects in.

We can pass that dictionary as a 
[display parameter](https://annotation.github.io/text-fabric/Api/App/#display)

In [23]:
highlights = {}

for a in sura2Ayas:
  firstWord = L.d(a, otype='word')[0]
  isVoc = F.posx.v(firstWord) == 'vocative'
  isRes = F.posx.v(firstWord) == 'resumption'
  if isVoc or isRes:
    highlights[firstWord] = 'lime' if isVoc else 'fuchsia'
  if a in startAyaNodes:
    highlights[a] = 'gold'
  elif a in endAyaNodes:
    highlights[a] = 'lightcoral'

### Color display of sections and particles

Ayas that start a section are marked with a bar in *gold*, the ones that end a section have a *light coral* bar.

Resumptive particles at the start of an aya are highlighted in *fuchsia*, the voactive ones in *lime*.

In [25]:
A.table(ayaTuples, start=240, end=260, highlights=highlights)

## Semantics

The article says: 

> interior sections correspond to each other: the second section corresponds to the second-to-last, and so on concentrically.

and

> By means of concentric patterning, ring composition calls attention to the center. We are drawn to look here for the essential message.

If you don't read Arabic, a translation is the next best way to access the semantics.

### Shuffled ayas

For each section, we will list the first three ayas (b1, b2, b3), the middle three (m1, m2, m3) and the last three (e1, e2, e3).

We produce them in the order b1 - e3 - b2 - e2 - b3 - e1 - m1 - m3 - m2.

We also put the first words, if they are particles of the vocative or resumption kind in the table.

Everything will be formatted in a markdown table.

In [26]:
def dm(markdownString):
  display(Markdown(markdownString))

### Construction of the table

We walk through the sections and grab all the information we need, and wrap it into a big
markdown table.

In [27]:
markdown = '''
section | aya | position | kind | particle | translation
--- | --- | --- | --- | --- | ---
'''.lstrip()

for (i, (beginAnum, endAnum)) in enumerate(sections):
  ayas = {}
  ayas['b1'] = T.nodeFromSection((2, beginAnum))
  ayas['e3'] = T.nodeFromSection((2, endAnum))
  ayas['m2'] = (ayas['b1'] + ayas['e3']) // 2
  ayas['b2'] = ayas['b1'] + 1
  ayas['b3'] = ayas['b1'] + 2
  ayas['m1'] = ayas['m2'] - 1
  ayas['m3'] = ayas['m2'] + 1
  ayas['e1'] = ayas['e3'] - 2
  ayas['e2'] = ayas['e3'] - 1
  for aName in 'b1 e3 b2 e2 b3 e1 m1 m3 m2'.split():
    a = ayas[aName]
    w = L.d(a, otype='word')[0]
    posx = F.posx.v(w)
    kind = posx if posx in {'vocative', 'resumption'} else None
    particle = None if kind is None else F.ascii.v(w)
    markdown += f'''
{i + 1} | {F.number.v(a)} | {aName} | {kind or '&nbsp;'} | {particle or '&nbsp;'} | {Fs('translation@en').v(a)}
'''.lstrip()

### Shuffled table display

In [28]:
dm(markdown)

section | aya | position | kind | particle | translation
--- | --- | --- | --- | --- | ---
1 | 1 | b1 | &nbsp; | &nbsp; | Alif Lam Mim
1 | 20 | e3 | &nbsp; | &nbsp; | the lightning wellnigh snatches away their sight; whensoever it gives them light, they walk in it, and when the darkness is over them, they halt; had God willed, He would have taken away their hearing and their sight. Truly, God is powerful over everything.
1 | 2 | b2 | &nbsp; | &nbsp; | That is the Book, wherein is no doubt, a guidance to the godfearing
1 | 19 | e2 | &nbsp; | &nbsp; | or as a cloudburst out of heaven in which is darkness, and thunder, and lightning -- they put their fingers in their ears against the thunderclaps, fearful of death; and God encompasses the unbelievers;
1 | 3 | b3 | &nbsp; | &nbsp; | who believe in the Unseen, and perform the prayer, and expend of that We have provided them;
1 | 18 | e1 | &nbsp; | &nbsp; | deaf, dumb, blind -- so they shall not return;
1 | 9 | m1 | &nbsp; | &nbsp; | They would trick God and the believers, and only themselves they deceive, and they are not aware.
1 | 11 | m3 | resumption | wa | When it is said to them, 'Do not corruption in the land', they say, 'We are only ones that put things right.'
1 | 10 | m2 | &nbsp; | &nbsp; | In their hearts is a sickness, and God has increased their sickness, and there awaits them a painful chastisement for that they have cried lies.
2 | 21 | b1 | vocative | ya`^ | O you men, serve your Lord Who created you, and those that were before you; haply so you will be godfearing;
2 | 39 | e3 | resumption | wa | As for the unbelievers who cry lies to Our signs, those shall be the inhabitants of the Fire, therein dwelling forever.'
2 | 22 | b2 | &nbsp; | &nbsp; | who assigned to you the earth for a couch, and heaven for an edifice, and sent down out of heaven water, wherewith He brought forth fruits for your provision; so set not up compeers to God wittingly.
2 | 38 | e2 | &nbsp; | &nbsp; | We said, 'Get you down out of it, all together; yet there shall come to you guidance from Me, and whosoever follows My guidance, no fear shall be on them, neither shall they sorrow.
2 | 23 | b3 | resumption | wa | And if you are in doubt concerning that We have sent down on Our servant, then bring a sura like it, and call your witnesses, apart from God, if you are truthful.
2 | 37 | e1 | resumption | fa | Thereafter Adam received certain words from his Lord, and He turned towards him; truly He turns, and is All-compassionate.
2 | 29 | m1 | &nbsp; | &nbsp; | It is He who created for you all that is in the earth, then He lifted Himself to heaven and levelled them seven heavens; and He has knowledge of everything.
2 | 31 | m3 | resumption | wa | And He taught Adam the names, all of them; then He presented them unto the angels and said, 'Now tell Me the names of these, if you speak truly.'
2 | 30 | m2 | resumption | wa | And when thy Lord said to the angels, 'I am setting in the earth a viceroy.' They said, 'What, wilt Thou set therein one who will do corruption there, and shed blood, while We proclaim Thy praise and call Thee Holy?' He said, 'Assuredly I know that you know not.'
3 | 40 | b1 | vocative | ya` | Children of Israel, remember My blessing wherewith I blessed you, and fulfil My covenant and I shall fulfil your covenant; and have awe of Me.
3 | 103 | e3 | resumption | wa | Yet had they believed, and been godfearing, a recompense from God had been better, if they had but known.
3 | 41 | b2 | &nbsp; | &nbsp; | And believe in that I have sent down, confirming that which is with you, and be not the first to disbelieve in it. And sell not My signs for a little price; and fear you Me.
3 | 102 | e2 | &nbsp; | &nbsp; | and they follow what the Satans recited over Solomon's kingdom. Solomon disbelieved not, but the Satans disbelieved, teaching the people sorcery, and that which was sent down upon Babylon's two angels, Harut and Marut; they taught not any man, without they said, 'We are but a temptation; do not disbelieve.' From them they learned how they might divide a man and his wife, yet they did not hurt any man thereby, save by the leave of God, and they learned what hurt them, and did not profit them, knowing well that whoso buys it shall have no share in the world to come; evil then was that they sold themselves for, if they had but known.
3 | 42 | b3 | &nbsp; | &nbsp; | And do not confound the truth with vanity, and do not conceal the truth wittingly.
3 | 101 | e1 | resumption | wa | When there has come to them a Messenger from God confirming what was with them, a party of them that were given the Book reject the Book of God behind their backs, as though they knew not,
3 | 70 | m1 | &nbsp; | &nbsp; | They said, 'Pray to thy Lord for us, that He make clear to us what she may be; cows are much alike to us; and, if God will, we shall then be guided.'
3 | 72 | m3 | &nbsp; | &nbsp; | And when you killed a living soul, and disputed thereon -- and God disclosed what you were hiding --
3 | 71 | m2 | &nbsp; | &nbsp; | He said, 'He says she shall be a cow not broken to plough the earth or to water the tillage, one kept secure, with no blemish on her.' They said, 'Now thou hast brought the truth'; and therefore they sacrificed her, a thing they had scarcely done.
4 | 104 | b1 | vocative | ya`^ | O believers, do not say, 'Observe us,' but say, 'Regard us'; and give ear; for unbelievers awaits a painful chastisement.
4 | 141 | e3 | &nbsp; | &nbsp; | That is a nation that has passed away; there awaits them that they have earned, and there awaits you that you have earned; you shall not be questioned concerning the things they did.
4 | 105 | b2 | &nbsp; | &nbsp; | Those unbelievers of the People of the Book and the idolaters wish not that any good should be sent down upon you from your Lord; but God singles out for His mercy whom He will; God is of bounty abounding.
4 | 140 | e2 | &nbsp; | &nbsp; | Or do you say, "Abraham, Ishmael, Isaac and Jacob, and the Tribes -- they were Jews, or they were Christians"?' Say: 'Have you then greater knowledge, or God? And who does greater evil than he who conceals a testimony received from God? And God is not heedless of the things you do.'
4 | 106 | b3 | &nbsp; | &nbsp; | And for whatever verse We abrogate or cast into oblivion, We bring a better or the like of it; knowest thou not that God is powerful over everything?
4 | 139 | e1 | &nbsp; | &nbsp; | Say: 'Would you then dispute with us concerning God, who is our Lord and your Lord? Our deeds belong to us, and to you belong your deeds; Him we serve sincerely.
4 | 121 | m1 | &nbsp; | &nbsp; | Those to whom We have given the Book and who recite it with true recitation, they believe in it; and whoso disbelieves in it, they shall be the losers.
4 | 123 | m3 | &nbsp; | &nbsp; | and beware a day when no soul for another shall give satisfaction, and no counterpoise shall be accepted from it, nor any intercession shall be profitable to it, neither shall they be helped.
4 | 122 | m2 | vocative | ya` | Children of Israel, remember My blessing wherewith I blessed you, and that I have preferred you above all beings;
5 | 142 | b1 | &nbsp; | &nbsp; | The fools among the people will say, 'What has turned them from the direction they were facing in their prayers aforetime?' Say: "To God belong the East and the West; He guides whomsoever He will to a straight path.'
5 | 152 | e3 | resumption | fa | So remember Me, and I will remember you; and be thankful to Me; and be you not ungrateful towards Me.
5 | 143 | b2 | resumption | wa | Thus We appointed you a midmost nation that you might be witnesses to the people, and that the Messenger might be a witness to you; and We did not appoint the direction thou wast facing, except that We might know who followed the Messenger from him who turned on his heels -- though it were a grave thing save for those whom God has guided; but God would never leave your faith to waste - truly, God is All-gentle with the people, All-compassionate.
5 | 151 | e2 | &nbsp; | &nbsp; | as also We have sent among you, of yourselves, a Messenger, to recite Our signs to you and to purify you, and to teach you the Book and the Wisdom, and to teach you that you knew not.
5 | 144 | b3 | &nbsp; | &nbsp; | We have seen thee turning thy face about in the heaven; now We will surely turn thee to a direction that shall satisfy thee. Turn thy face towards the Holy Mosque; and wherever you are, turn your faces towards it. Those who have been given the Book know it is the truth from their Lord; God is not heedless of the things they do.
5 | 150 | e1 | resumption | wa | From whatsoever place thou issuest, turn thy face towards the Holy Mosque; and wherever you may be, turn your faces towards it, that the people may not have any argument against you, excepting the evildoers of them; and fear you them not, but fear you Me; and that I may perfect My blessing upon you, and that haply so you may be guided;
5 | 146 | m1 | &nbsp; | &nbsp; | whom We have given the Book, and they recognize as they recognize their sons, even though there is a party of them conceal the truth and that wittingly.
5 | 148 | m3 | resumption | wa | Every man has his direction to which he turns; so be you forward in good works. Wherever you may be, God will bring you all together; surely God is powerful over everything.
5 | 147 | m2 | &nbsp; | &nbsp; | The truth comes from thy Lord; then be not among the doubters.
6 | 153 | b1 | vocative | ya`^ | O all you who believe, seek you help in patience and prayer; surely God is with the patient.
6 | 177 | e3 | &nbsp; | &nbsp; | It is not piety, that you turn your faces to the East and to the West. True piety is this: to believe in God, and the Last Day, the angels, the Book, and the Prophets, to give of one's substance, however cherished, to kinsmen, and orphans, the needy, the traveller, beggars, and to ransom the slave, to perform the prayer, to pay the alms. And they who fulfil their covenant when they have engaged in a covenant, and endure with fortitude misfortune, hardship and peril, these are they who are true in their faith, these are the truly godfearing.
6 | 154 | b2 | resumption | wa | And say not of those slain in God's way, 'They are dead'; rather they are living, but you are not aware.
6 | 176 | e2 | &nbsp; | &nbsp; | That, because God has sent down the Book with the truth; and those that are at variance regarding the Book are in wide schism.
6 | 155 | b3 | resumption | wa | Surely We will try you with something of fear and hunger, and diminution of goods and lives and fruits; yet give thou good tidings unto the patient
6 | 175 | e1 | &nbsp; | &nbsp; | Those are they that have bought error at the price of guidance, and chastisement at the price of pardon; how patiently they shall endure the Fire!
6 | 164 | m1 | &nbsp; | &nbsp; | .. Surely in the creation of the heavens and the earth and the alternation of night and day and the ship that runs in the sea with profit to men, and the water God sends down from heaven therewith reviving the earth after it is dead and His scattering abroad in it all manner of crawling thing, and the turning about of the winds and the clouds compelled between heaven and earth -- surely there are signs for a people having understanding.
6 | 166 | m3 | &nbsp; | &nbsp; | when those that were followed disown their followers, and they see the chastisement, and their cords are cut asunder,
6 | 165 | m2 | resumption | wa | Yet there be men who take to themselves compeers apart from God, loving them as God is loved; but those that believe love God more ardently. O if the evildoers might see, when they see the chastisement, that the power altogether belongs to God, and that God is terrible in chastisement,
7 | 178 | b1 | vocative | ya`^ | O believers, prescribed for you is retaliation, touching the slain; freeman for freeman, slave for slave, female for female. But if aught is pardoned a man by his brother, let the pursuing be honourable, and let the payment be with kindliness. That is a lightening granted you by your Lord, and a mercy; and for him who commits aggression after that -- for him there awaits a painful chastisement.
7 | 253 | e3 | &nbsp; | &nbsp; | And those Messengers, some We have preferred above others; some there are to whom God spoke, and some He raised in rank. And We gave Jesus son of Mary the clear signs, and confirmed him with the Holy Spirit. And had God willed, those who came after him would not have fought one against the other after the clear signs had come to them; but they fell into variance, and some of them believed, and some disbelieved; and had God willed they would not have fought one against the other; but God does whatsoever He desires.
7 | 179 | b2 | resumption | wa | In retaliation there is life for you, men possessed of minds; haply you will be godfearing.
7 | 252 | e2 | &nbsp; | &nbsp; | These are the signs of God We recite to thee in truth, and assuredly thou art of the number of the Envoys.
7 | 180 | b3 | &nbsp; | &nbsp; | Prescribed for you, when any of you is visited by death, and he leaves behind some goods, is to make testament in favour of his parents and kinsmen honourably -- an obligation on the godfearing.
7 | 251 | e1 | resumption | fa | And they routed them, by the leave of God, and David slew Goliath; and God gave him the kingship, and Wisdom, and He taught him such as He willed. Had God not driven back the people, some by the means of others, the earth had surely corrupted; but God is bounteous unto all beings.
7 | 214 | m1 | &nbsp; | &nbsp; | Or did you suppose you should enter Paradise without there had come upon you the like of those who passed away before you? They were afflicted by misery and hardship and were so convulsed, that the Messenger and those who believed with him said, 'When comes God's help?' Ah, but surely God's help is nigh.
7 | 216 | m3 | &nbsp; | &nbsp; | Prescribed for you is fighting, though it be hateful to you. Yet it may happen that you will hate a thing which is better for you; and it may happen that you will love a thing which is worse for you; God knows, and you know not.
7 | 215 | m2 | &nbsp; | &nbsp; | They will question thee concerning what they should expend. Say: 'Whatsoever good you expend is for parents and kinsmen, orphans, the needy, and the traveller; and whatever good you may do, God has knowledge of it.'
8 | 254 | b1 | vocative | ya`^ | O believers, expend of that wherewith We have provided you, before there comes a day wherein shall be neither traffick, nor friendship, nor intercession; and the unbelievers -- they are the evildoers.
8 | 284 | e3 | &nbsp; | &nbsp; | To God belongs all that is in the heavers and earth. Whether you publish what is in your hearts or hide it, God shall make reckoning with you for it. He will forgive whom He will, and chastise whom He will; God is powerful over everything.
8 | 255 | b2 | &nbsp; | &nbsp; | God there is no god but He, the Living, the Everlasting. Slumber seizes Him not, neither sleep; to Him belongs all that is in the heavens and the earth. Who is there that shall intercede with Him save by His leave? He knows what lies before them and what is after them, and they comprehend not anything of His knowledge save such as He wills. His Throne comprises the heavens and earth; the preserving of them oppresses Him not; He is the All-high, the All-glorious.
8 | 283 | e2 | resumption | wa | And if you are upon a journey, and you do not find a writer, then a pledge in hand. But if one of you trusts another, let him who is trusted deliver his trust, and let him fear God his Lord. And do not conceal the testimony; whoso conceals it, his heart is sinful; and God has knowledge of the things you do.
8 | 256 | b3 | &nbsp; | &nbsp; | No compulsion is there in religion. Rectitude has become clear from error. So whosoever disbelieves in idols and believes in God, has laid hold of the most firm handle, unbreaking; God is All-hearing, All-knowing.
8 | 282 | e1 | vocative | ya`^ | O believers, when you contract a debt one upon another for a stated term, write it down, and let a writer write it down between you justly, and let not any writer refuse to write it down, as God has taught him; so let him write, and let the debtor dictate, and let him fear God his Lord and not diminish aught of it. And if the debtor be a fool, or weak, or unable to dictate himself, then let his guardian dictate justly. And call in to witness two witnesses, men; or if the two be not men, then one man and two women, such witnesses as you approve of, that if one of the two women errs the other will remind her; and let the witnesses not refuse, whenever they are summoned. And be not loth to write it down, whether it be small or great, with its term; that is more equitable in God's sight, more upright for testimony, and likelier that you will not be in doubt. Unless it be merchandise present that you give and take between you; then it shall be no fault in you if you do not write it down. And take witnesses when you are trafficking one with another. And let not either writer or witness be pressed; or if you do, that is ungodliness in you. And fear God; God teaches you, and God has knowledge of everything.
8 | 268 | m1 | &nbsp; | &nbsp; | Satan promises you poverty, and bids you unto indecency; but God promises you His pardon and His bounty; and God is All-embracing, All-knowing.
8 | 270 | m3 | resumption | wa | And whatever expenditure you expend, and whatever vow you vow, surely God knows it. No helpers have the evildoers.
8 | 269 | m2 | &nbsp; | &nbsp; | He gives the Wisdom to whomsoever He will, and whoso is given the Wisdom, has been given much good; yet none remembers but men possessed of minds.
9 | 285 | b1 | &nbsp; | &nbsp; | The Messenger believes in what was sent down to him from his Lord, and the believers; each one believes in God and His angels, and in His Books and His Messengers; we make no division between any one of His Messengers. They say, 'We hear, and obey. Our Lord, grant us Thy forgiveness; unto Thee is the homecoming.'
9 | 286 | e3 | &nbsp; | &nbsp; | God charges no soul save to its capacity; standing to its account is what it has earned, and against its account what it has merited. Our Lord, take us not to task if we forget, or make mistake. Our Lord; charge us not with a load such as Thou didst lay upon those before us. Our Lord, do Thou not burden us beyond what we have the strength to bear. And pardon us, and forgive us, and have mercy on us; Thou art our Protector. And help us against the people of the unbelievers.
9 | 286 | b2 | &nbsp; | &nbsp; | God charges no soul save to its capacity; standing to its account is what it has earned, and against its account what it has merited. Our Lord, take us not to task if we forget, or make mistake. Our Lord; charge us not with a load such as Thou didst lay upon those before us. Our Lord, do Thou not burden us beyond what we have the strength to bear. And pardon us, and forgive us, and have mercy on us; Thou art our Protector. And help us against the people of the unbelievers.
9 | 285 | e2 | &nbsp; | &nbsp; | The Messenger believes in what was sent down to him from his Lord, and the believers; each one believes in God and His angels, and in His Books and His Messengers; we make no division between any one of His Messengers. They say, 'We hear, and obey. Our Lord, grant us Thy forgiveness; unto Thee is the homecoming.'
9 | 1 | b3 | &nbsp; | &nbsp; | Alif Lam Mim
9 | 284 | e1 | &nbsp; | &nbsp; | To God belongs all that is in the heavers and earth. Whether you publish what is in your hearts or hide it, God shall make reckoning with you for it. He will forgive whom He will, and chastise whom He will; God is powerful over everything.
9 | 284 | m1 | &nbsp; | &nbsp; | To God belongs all that is in the heavers and earth. Whether you publish what is in your hearts or hide it, God shall make reckoning with you for it. He will forgive whom He will, and chastise whom He will; God is powerful over everything.
9 | 286 | m3 | &nbsp; | &nbsp; | God charges no soul save to its capacity; standing to its account is what it has earned, and against its account what it has merited. Our Lord, take us not to task if we forget, or make mistake. Our Lord; charge us not with a load such as Thou didst lay upon those before us. Our Lord, do Thou not burden us beyond what we have the strength to bear. And pardon us, and forgive us, and have mercy on us; Thou art our Protector. And help us against the people of the unbelievers.
9 | 285 | m2 | &nbsp; | &nbsp; | The Messenger believes in what was sent down to him from his Lord, and the believers; each one believes in God and His angels, and in His Books and His Messengers; we make no division between any one of His Messengers. They say, 'We hear, and obey. Our Lord, grant us Thy forgiveness; unto Thee is the homecoming.'


# Sentiment Mining (primitive)

As the article points out, sections often start by speaking to *good* people or actions, and end with condemning *bad*
people or actions. Can we base sentiment mining on this idea?

## Positivity located in the rings

If a verb, noun or adjective occurs in an e1, e2, or e3 aya, its positivity is deccreased by 1, if it occurs in a b1, b2, b3 it is increased by 1.

This is very crude, because we have taken the ring structure very rigidly.
In the actual text, there might be ayas that are less pertinent to the ring structure, and the middle of the ring 
does not have to coincide with the exact middle aya of the section.

It is up to the expert to devise additional heuristics that help to identify the rings in a more precise mannner.

## Calculating positivity

Now we are going to assign a positivity to all lemmas (verb, noun or adjective) of the sura.

In [29]:
positivity = collections.Counter()
contentCats = {'verb', 'noun', 'word'}

for (i, (beginAnum, endAnum)) in enumerate(sections):
  ayas = {}
  ayas['b1'] = T.nodeFromSection((2, beginAnum))
  ayas['e3'] = T.nodeFromSection((2, endAnum))
  ayas['m2'] = (ayas['b1'] + ayas['e3']) // 2
  ayas['b2'] = ayas['b1'] + 1
  ayas['b3'] = ayas['b1'] + 2
  ayas['m1'] = ayas['m2'] - 1
  ayas['m3'] = ayas['m2'] + 1
  ayas['e1'] = ayas['e3'] - 2
  ayas['e2'] = ayas['e3'] - 1
  for aName in 'b1 e3 b2 e2 b3 e1 m1 m3 m2'.split():
    a = ayas[aName]
    pos = 1 if aName.startswith('b') else -1 if aName.startswith('e') else 0
    for w in L.d(a, otype='word'):
      if F.pos.v(w) in contentCats:
        positivity[F.lemma.v(w)] += pos

## Table of lemmas and their positivity

We show the positivity of the lemmas of this sura.

In [30]:
for (lemma, pos) in sorted(
  positivity.items(),
  key=lambda x: (-x[1], x[0]),
):
  print(f'{lemma:<20} has positivity {pos:>2}')

>ay~uhaA             has positivity  4
'aAmana              has positivity  3
Eabod                has positivity  3
jaEala               has positivity  3
qibolap              has positivity  3
>anfaqa              has positivity  2
>ataY                has positivity  2
>awofaY`             has positivity  2
>unvaY`              has positivity  2
Hay~                 has positivity  2
Hur~                 has positivity  2
hadaY                has positivity  2
mivol                has positivity  2
m~aEoruwf            has positivity  2
qabol                has positivity  2
qiSaAS               has positivity  2
raHomap              has positivity  2
rayob                has positivity  2
razaqa               has positivity  2
samaA^'              has positivity  2
vamara`t             has positivity  2
xayor                has positivity  2
$afa`Eap             has positivity  1
<iHosa`n             has positivity  1
<ikoraAh             has positivity  1
<ila`h               has 

## Coloring the sura

Finally, we display the whole sura with words colored according to their positivity or negativity.



### Color gradients

Negative words marked red, and postive ones marked green.
The farther from zero, the darker the color.

The 
[HSL scheme](https://developer.mozilla.org/en-US/docs/Web/CSS/color_value#HSL_colors)
of color specification is very convenient here.

In [31]:
def posColor(p):
  if p <= -20:
    return 'hsl(0, 100%, 50%)'
  if p <= -5:
    return 'hsl(0, 90%, 60%)'
  if p <= -3:
    return 'hsl(0, 80%, 70%)'
  if p == -2:
    return 'hsl(30, 80%, 70%)'
  if p == -1:
    return 'hsl(60, 80%, 70%)'
  if p == 0:
    return 'hsl(90, 70%, 80%)'
  if p == 1:
    return 'hsl(120, 80%, 70%)'
  if p == 2:
    return 'hsl(120, 90%, 80%)'
  if p == 3:
    return 'hsl(120, 100%, 90%)'
  if p == 4:
    return 'hsl(120, 100%, 100%)'
  if p >= 4:
    return 'hsl(150, 100%, 100%)'

### Computing highlights

We just compute a highlight mapping, much like we did before, but now we are going to highlight
all words in the sura that have a positivity or negativity.

In [32]:
startAyaNodes = set()
endAyaNodes = set()

for (beginAnum, endAnum) in sections:
  beginA = T.nodeFromSection((2, beginAnum))
  endA = T.nodeFromSection((2, endAnum))
  startAyaNodes.add(beginA)
  endAyaNodes.add(endA)
highlights = {}

for a in sura2Ayas:
  words = L.d(a, otype='word')
  firstWord = words[0]
  isVoc = F.posx.v(firstWord) == 'vocative'
  isRes = F.posx.v(firstWord) == 'resumption'
  for w in words:
    if w == firstWord:
      if isVoc or isRes:
        highlights[firstWord] = 'lime' if isVoc else 'fuchsia'
    if F.lemma.v(w) in positivity:
      pos = positivity[F.lemma.v(w)]
      highlights[w] = posColor(pos)
  if a in startAyaNodes:
    highlights[a] = 'gold'
  elif a in endAyaNodes:
    highlights[a] = 'lightcoral'

### Colored sentiment display

In [33]:
A.table(ayaTuples, start=240, end=260, highlights=highlights)

# Saving data

The real power of TF comes now: we can save the sentiments as a new feature and use it as if it were given with the data set.

We save it to a temporary location, in your downloads folder, and load it from there.

We use
[TF.save()](https://annotation.github.io/text-fabric/Api/Fabric/#saving-features) to save the feature.

## Gathering the parameters

We set up all data that `TF.save()` needs.

In [34]:
metaData = {
  'sentiment': {
    'valueType': 'int',
    'description': 'crude sentiment values for some words in sura 2',
  },
}

edgeFeatures = {}
nodeFeatures = {
  'sentiment': {w: positivity[F.lemma.v(w)] for w in L.d(sura2, otype='word') if F.lemma.v(w) in positivity},
}

location = os.path.expanduser('~/Downloads/q-ran/quran/tf')
module = A.version

## Performing the save

Now we do the save action:

In [35]:
TF.save(metaData=metaData, edgeFeatures=edgeFeatures, nodeFeatures=nodeFeatures, location=location, module=module)

True

## Checking

Let's check (maybe this does not work on Windows. Open the file explorer and locate the file yourself.

In [36]:
%%sh
ls -lR ~/Downloads/q-ran/quran/tf

total 0
drwxr-xr-x  3 dirk  staff  96 May  8 12:49 0.4

/Users/dirk/Downloads/q-ran/quran/tf/0.4:
total 40
-rw-r--r--  1 dirk  staff  19703 May  8 12:49 sentiment.tf


# Using new data

We are going to load the Quran data again plus the sentiment data

## Loading the sentiment data

We use roughly the same statement as when we first loaded the data,
but now we ask TF also to look into the directory where we saved the sentiments.

We do that by supplying the `locations` and `modules` parameters.

See also
[A.use()](https://annotation.github.io/text-fabric/Api/App/#incantation)

In [37]:
# A = use('quran', hoist=globals(), locations=location, modules=module)
A = use('quran:clone', checkout="clone", hoist=globals(), locations=location, modules=module)

Using TF-app in /Users/dirk/github/annotation/app-quran/code:
	repo clone offline under ~/github (local github)
Using data in /Users/dirk/github/q-ran/quran/tf/0.4:
	repo clone offline under ~/github (local github)


See: the feature `sentiment` got loaded.

## Inspecting the sentiment data

Let's inspect it.

In [38]:
F.sentiment.freqList()

((-1, 710),
 (0, 621),
 (1, 475),
 (-2, 317),
 (-21, 282),
 (2, 159),
 (3, 89),
 (-4, 79),
 (-3, 62),
 (-7, 28),
 (-5, 21),
 (4, 13))

or, a bit more pleasing

In [39]:
for (sent, amount) in F.sentiment.freqList():
  print(f'sentiment {sent:>3} occurs {amount:>3} times')

sentiment  -1 occurs 710 times
sentiment   0 occurs 621 times
sentiment   1 occurs 475 times
sentiment  -2 occurs 317 times
sentiment -21 occurs 282 times
sentiment   2 occurs 159 times
sentiment   3 occurs  89 times
sentiment  -4 occurs  79 times
sentiment  -3 occurs  62 times
sentiment  -7 occurs  28 times
sentiment  -5 occurs  21 times
sentiment   4 occurs  13 times


## Displaying sentiment data

Remember the display where we showed the morphology and the translations?

We can make the same display, but now with sentiments shown.

We show only some of the ayas.

We have to add the feature `sentiment` to the features to be displayed.

In [40]:
A.displaySetup(extraFeatures='translation@en sentiment')
A.show(ayaTuples, start=10, end=20)

## Searching for sentiment data

We can use the sentiment feature in searches.

Let's look for negative words in ayas starting with a vocative particle.

In [41]:
query = '''
sura number=2
  aya
    =: word posx=vocative
    word sentiment<0
'''

In [42]:
results = A.search(query)

  0.25s 107 results


Before we show the results, lets furnish an uncluttered
[display](https://annotation.github.io/text-fabric/Api/App/#display)
by suppressing a number of features.

In [47]:
A.displaySetup(extraFeatures='translation@en', suppress="number posx")

We want to see multiple negative words of an aya highlighted in the same aya, instead of displaying the aya over and over again for each
negative word. Hence we'll pass `condensed=True`.

In [49]:
A.show(results, start=1, end=5, condensed=True)

# Export to Excel

It is convenient to export this to Excel, for more intense inspection, aided by all the tools that Excel has to offer for sorting, filtering and making charts.

We use the function 
[A.export()]()
to do this.

This function takes a list of node tuples, retrieves data about those nodes, and writes it to a .tsv file (tab-separated), that
can be opened by Excel (even with Arabic characters).

In [50]:
A.export(results)

The result is now in your Downloads folder, with name `results.tsv`

In [51]:
%%sh
ls -l ~/Downloads/results.tsv
head -n 10 ~/Downloads/results.tsv

-rw-r--r--@ 1 dirk  staff  204326 May  8 12:52 /Users/dirk/Downloads/results.tsv
��R 	 S 1 	 S 2 	 N O D E 1 	 T Y P E 1 	 n u m b e r 1 	 N O D E 2 	 T Y P E 2 	 T E X T 2 	 N O D E 3 	 T Y P E 3 	 T E X T 3 	 p o s x 3 	 N O D E 4 	 T Y P E 4 	 T E X T 4 	 s e n t i m e n t 4 
 1 	 2 	 2 1 	 2 1 8 1 7 0 	 s u r a 	 2 	 1 2 8 2 4 7 	 a y a 	 JNpS#NJQOGN'  qDFQN'3O  q9R(O/OH'�  1N(QNCOEO  qDQN0PI  .NDNBNCOER  HNqDQN0PJFN  EPF  BN(RDPCOER  DN9NDQNCOER  *N*QNBOHFN	 4 3 3 	 w o r d 	 JNpS	 v o c a t i v e 	 4 3 9 	 w o r d 	 1N(QN	 - 1 
 2 	 2 	 2 1 	 2 1 8 1 7 0 	 s u r a 	 2 	 1 2 8 2 4 7 	 a y a 	 JNpS#NJQOGN'  qDFQN'3O  q9R(O/OH'�  1N(QNCOEO  qDQN0PI  .NDNBNCOER  HNqDQN0PJFN  EPF  BN(RDPCOER  DN9NDQNCOER  *N*QNBOHFN	 4 3 3 	 w o r d 	 JNpS	 v o c a t i v e 	 4 5 1 	 w o r d

Not very pretty, because the file is in utf16 and the terminal does not handle that well.
But this is the encoding that Excel can use, and there it looks allright:

![nega](images/negativeExcel.png)

# Share your data

You can share the new `sentiment` feature in such a way that others can use it easily, even more easy than we just did.

The thing to do is:

* use text-fabric to store your features in a zip file
* make or use a github repository
* create a new release and add the zip file to it

These steps are explained in the [share](share.ipynb) tutorial.
There we made an even cruder feature `sentiment`, with words in all suras, and stored it on GitHub
in
[q-ran/exercises/mining/tf](https://github.com/q-ran/exercises/tree/master/mining/tf/0.3).

# Re-use your data

That means that we can call up that data alongside the Quran data by passing an extra argument to `use()`:

In [52]:
# A = use('quran', hoist=globals(), mod='q-ran/exercises/mining/tf')
A = use('quran:clone', checkout="clone", hoist=globals(), mod='q-ran/exercises/mining/tf:clone')

Using TF-app in /Users/dirk/github/annotation/app-quran/code:
	repo clone offline under ~/github (local github)
Using data in /Users/dirk/github/q-ran/quran/tf/0.4:
	repo clone offline under ~/github (local github)
Using data in /Users/dirk/github/q-ran/exercises/mining/tf/0.4:
	repo clone offline under ~/github (local github)


Note that you can now click on the `sentiment` feature above, which will bring you to
the repository where it is stored.
If the creator of the data did a good job, you'll find documentation about `sentiment` in its readme file
or docs directory.

We check the feature itself for its distribution.

In [53]:
F.sentiment.freqList()

((-1, 4509),
 (-2, 3044),
 (-233, 2699),
 (-4, 2368),
 (-3, 2217),
 (1, 2040),
 (-5, 1964),
 (-7, 1693),
 (-195, 1618),
 (-8, 1541),
 (0, 1517),
 (-6, 1416),
 (-10, 1406),
 (-138, 1358),
 (-49, 1147),
 (-130, 975),
 (-32, 865),
 (-21, 841),
 (2, 833),
 (-13, 696),
 (-12, 690),
 (-14, 669),
 (-36, 662),
 (-25, 589),
 (-17, 589),
 (-9, 577),
 (-30, 571),
 (29, 537),
 (-22, 529),
 (-11, 431),
 (-23, 410),
 (-18, 360),
 (-45, 358),
 (-33, 289),
 (-54, 278),
 (-37, 271),
 (-31, 271),
 (-24, 261),
 (-16, 245),
 (-26, 236),
 (3, 205),
 (-73, 176),
 (122, 153),
 (-34, 136),
 (-39, 127),
 (5, 115),
 (-20, 75),
 (11, 75),
 (-15, 72),
 (4, 49))

Note that you can load as many extra modules as you want. If one researcher adds semantic domains, another sentiments, yet another named
entities, then everybody can use semantics, sentiments and entities in one go.

---

All chapters:

* **[start](start.ipynb)** introduction to computing with your corpus
* **[display](display.ipynb)** become an expert in creating pretty displays of your text structures
* **[search](search.ipynb)** turbo charge your hand-coding with search templates
* **[exportExcel](exportExcel.ipynb)** make tailor-made spreadsheets out of your results
* **[share](share.ipynb)** draw in other people's data and let them use yours
* **[similarAyas](similarAyas.ipynb)** spot the similarities between lines
* **rings** ring structures in sura 2

CC-BY Dirk Roorda